In [4]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
from smef.core.drives.types import DriveSpec
from smef.core.units import Q
from smef.engine import SimulationEngine, UnitSystem

from bec.light.classical import carrier_profiles
from bec.light.classical.carrier import Carrier
from bec.light.classical.factories import gaussian_field_drive
from bec.light.classical.field_drive import ClassicalFieldDriveU
from bec.light.core.polarization import JonesState
from bec.quantum_dot.dot import QuantumDot
from bec.quantum_dot.enums import QDState, TransitionPair
from bec.quantum_dot.smef.initial_state import rho0_qd_vacuum
from bec.quantum_dot.spec.dipole_params import DipoleParams
from bec.quantum_dot.spec.energy_structure import EnergyStructure
from bec.quantum_dot.spec.phonon_params import (
    PhenomenologicalPhononParams,
    PhononModelType,
    PhononParams,
    PolaronPhononParams,
)
from bec.reporting.plotting.api import plot_runs
from bec.reporting.plotting.grid import PlotConfig

In [2]:
phonons = PhononParams(
    model=PhononModelType.POLARON,
    temperature=Q(4.0, "K"),
    phenomenological=PhenomenologicalPhononParams(
        gamma_phi_Xp=Q(1.0e9, "1/s"),
        gamma_phi_Xm=Q(1.0e9, "1/s"),
        gamma_phi_XX=Q(0.0, "1/s"),
    ),
    polaron=PolaronPhononParams(
        enable_polaron_renorm=True,
        alpha=Q(0.03, "ps**2"),
        omega_c=Q(1.0e12, "rad/s"),
        enable_exciton_relaxation=False,
    ),
)

energy = EnergyStructure(
    X1=Q(1.201, "eV"),
    X2=Q(1.201, "eV"),
    XX=Q(2.600, "eV"),
)

dipoles = DipoleParams(mu_default=Q(1e-27, "C*m"))

qd = QuantumDot(energy=energy, dipoles=dipoles, phonons=phonons)

time_unit_s = float(Q(1.0, "ps").to("s").magnitude)
units = UnitSystem(time_unit_s=time_unit_s)

engine = SimulationEngine(audit=True)

tlist = np.linspace(0.0, 450.0, 4501)


In [7]:
def _carrier_resonant(*, omega_target_rad_s: float) -> Carrier:
    return Carrier(
        omega0=Q(float(omega_target_rad_s), "rad/s"),
        delta_omega=carrier_profiles.constant(Q(0.0, "rad/s")),
    )


def _make_gaussian_drive(
    *,
    label: str,
    omega_target_rad_s: float,
    t0_ps: float,
    sigma_ps: float,
    E0_v_m: float,
    preferred_kind: str,
    pol_state=None,
) -> ClassicalFieldDriveU:
    base = gaussian_field_drive(
        t0=Q(float(t0_ps), "ps"),
        sigma=Q(float(sigma_ps), "ps"),
        E0=Q(float(E0_v_m), "V/m"),
        energy=Q(1.3, "eV"),
        delta_omega=Q(0.0, "rad/s"),
        pol_state=pol_state,
        preferred_kind=preferred_kind,
        label=label,
    )

    carrier = _carrier_resonant(omega_target_rad_s=omega_target_rad_s)

    return ClassicalFieldDriveU(
        envelope=base.envelope,
        amplitude=base.amplitude,
        carrier=carrier,
        pol_state=base.pol_state,
        pol_transform=base.pol_transform,
        preferred_kind=base.preferred_kind,
        label=label,
    )


def run_once(*, drive: ClassicalFieldDriveU):
    bundle = qd.compile_bundle(units=units)
    dims = bundle.modes.dims()
    rho0 = rho0_qd_vacuum(dims=dims, qd_state=QDState.G)

    specs = [DriveSpec(payload=drive, drive_id=drive.label)]

    solve_options = {
        "qutip_options": {
            "method": "bdf",
            "atol": 1e-10,
            "rtol": 1e-8,
            "nsteps": 200000,
            "max_step": 0.02,
            "progress_bar": "tqdm",
        }
    }

    return engine.run(
        qd,
        tlist=tlist,
        time_unit_s=time_unit_s,
        rho0=rho0,
        drives=specs,
        solve_options=solve_options,
    )

In [5]:

omega_G_X1 = float(qd.derived.omega_ref_rad_s(TransitionPair.G_X1))
omega_G_X2 = float(qd.derived.omega_ref_rad_s(TransitionPair.G_X2))
omega_G_XX_half = 0.5 * float(qd.derived.omega_ref_rad_s(TransitionPair.G_XX))

drive_H = _make_gaussian_drive(
    label="H_pulse",
    omega_target_rad_s=omega_G_X1,
    t0_ps=140.0,
    sigma_ps=25.0,
    E0_v_m=6e3,
    preferred_kind="1ph",
    pol_state=JonesState.H,  # set to your H state if available
)

drive_V = _make_gaussian_drive(
    label="V_pulse",
    omega_target_rad_s=omega_G_X2,
    t0_ps=240.0,
    sigma_ps=25.0,
    E0_v_m=6e3,
    preferred_kind="1ph",
    pol_state=JonesState.V,  # set to your V state if available
)

drive_XX = _make_gaussian_drive(
    label="XX_pulse",
    omega_target_rad_s=omega_G_XX_half,
    t0_ps=340.0,
    sigma_ps=25.0,
    E0_v_m=8e3,
    preferred_kind="2ph",
    pol_state=JonesState.D,
)


In [6]:

res_H = run_once(drive=drive_H)
res_V = run_once(drive=drive_V)
res_XX = run_once(drive=drive_XX)

figs = plot_runs(
    [res_H, res_V, res_XX],
    units=units,
    drives=[drive_H, drive_V, drive_XX],
    qds=[qd, qd, qd],
    cfg=PlotConfig(
        title="Three scenarios: H then V then XX (no chirp, separate runs)",
        ncols=3,
        column_titles=["H pulse (G->X1)", "V pulse (G->X2)", "XX pulse (2ph G->XX)"],
        show_omega_L=True,
        show_coupling_panel=True,
        coupling_mode="abs",
        sharex=True,
        sharey_by_row=True,
    ),
)

for fig in figs:
    fig.show()

plt.show()


rates keys: ['RAD_XX_X1', 'RAD_XX_X2', 'RAD_X1_G', 'RAD_X2_G', 'PH_DEPH_X1', 'PH_DEPH_X2']
PH_DEPH_X1 lookup: 1000000000.0 / second 1000000000.0 / second
PH_DEPH_X2 lookup: 1000000000.0 / second 1000000000.0 / second

rates keys: ['RAD_XX_X1', 'RAD_XX_X2', 'RAD_X1_G', 'RAD_X2_G', 'PH_DEPH_X1', 'PH_DEPH_X2']
PH_DEPH_X1 lookup: 1000000000.0 / second 1000000000.0 / second
PH_DEPH_X2 lookup: 1000000000.0 / second 1000000000.0 / second



AttributeError: 'function' object has no attribute 'as_array'